# Collect teams' ranking and matches history data

## Teams' ranking points

In [11]:
with open('../data/ranking.html', encoding='utf-8') as file:
    content = file.read().strip('\n').split('#')

pts = []

for team in content[1:]:

    info = []

    index_pts = team.index(' pts')
    info.append(int(team[team.rindex(' ', 0, index_pts) : index_pts]))

    team = team[index_pts + 5 : -1]
    
    # remove region from team name
    if team.find('CIS') == 0 or team.find('SEA') == 0:
        team = team[3:]
    else:
        team = team[2:]

    info.append(team)

    pts.append(info)

## Matches history

In [10]:
rank

[[1007, 'Team Secret'],
 [420, 'Vici Gaming'],
 [409, 'Virtus.pro'],
 [408, 'Team Liquid'],
 [381, 'Evil Geniuses'],
 [324, 'Team Nigma'],
 [266, 'Alliance'],
 [234, 'OG'],
 [217, 'PSG.LGD'],
 [215, 'Invictus\nGaming'],
 [210, 'VP.Prodigy'],
 [207, 'Ninjas in Pyjamas'],
 [203, 'Fnatic'],
 [191, 'EHOME'],
 [150, 'Team Spirit'],
 [143, 'HellRaisers'],
 [139, 'Vikin.gg'],
 [132, 'Natus Vincere'],
 [127, 'TNC\nPredator'],
 [118, 'Royal Never Give Up'],
 [114, 'Team Aster'],
 [83, 'Sparking Arrow Gaming'],
 [72, 'Quincy Crew'],
 [66, 'FlyToMoon'],
 [66, 'Newbee'],
 [65, 'beastcoast'],
 [60, 'OG Seed'],
 [50, 'CR4ZY'],
 [37, 'Gambit Esports'],
 [28, 'Geek Fam'],
 [27, 'Reality Rift'],
 [27, 'Former Players of paiN Gaming'],
 [26, 'CDEC Gaming'],
 [24, 'Thunder Predator'],
 [20, 'Team Unique'],
 [14, 'Keen Gaming'],
 [13, 'B8'],
 [10, 'BOOM Esports'],
 [9, 'business associates'],
 [9, 'Aggressive Mode'],
 [9, 'Cyber Legac']]